In [7]:
#This notebook is used to predict the survivors in Titanic according to the different features avaiable for each person.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [8]:
#add in dataset
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
all_data = pd.concat([train,test])

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [10]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [11]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [12]:
df_num = train[['Age','SibSp','Parch','Fare']]
df_cat = train[['PassengerId','Survived','Pclass','Sex','Cabin','Embarked','Ticket']]

In [13]:
df_num.columns

Index(['Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [14]:
# for i in df_num.columns:
#     plt.hist(df_num[i])
#     plt.title(i)
#     plt.show()

In [15]:
pd.pivot_table(train,index='Survived',values=['Fare','Age','Parch','SibSp','Pclass'])

,Age,Fare,Parch,Pclass,SibSp
Survived,,,,,
0,30.626179,22.117887,0.329690,2.531876,0.553734
1,28.343690,48.395408,0.464912,1.950292,0.473684


In [16]:
train.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.036847,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,0.036847,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,-0.057527,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000


In [17]:
# for i in df_cat.columns:
#     sns.barplot(df_cat[i].value_counts().index,df_cat[i].value_counts())
#     plt.title(i)
#     plt.show()

In [18]:
#Let's do some feature engineering and remove unnecessary features
train['CabinMultiple'] = train['Cabin'].apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))
train['Title'] = train['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())
list_1 = train['Title'].value_counts().index
list_2 = list_1.difference(['Mr','Mrs','Miss','Master'])
train.drop(columns=['Name','Cabin','Ticket'],inplace=True)
train['Title'] = train['Title'].replace(list_2,'Others')
train.dropna(subset=['Embarked'],inplace=True)
# train.Age = train.Age.fillna(train.Age.median)

In [19]:
train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,CabinMultiple,Title
0,1,0,3,male,22.0,1,0,7.2500,S,0,Mr
1,2,1,1,female,38.0,1,0,71.2833,C,1,Mrs
2,3,1,3,female,26.0,0,0,7.9250,S,0,Miss
3,4,1,1,female,35.0,1,0,53.1000,S,1,Mrs
4,5,0,3,male,35.0,0,0,8.0500,S,0,Mr
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,13.0000,S,0,Others
887,888,1,1,female,19.0,0,0,30.0000,S,1,Miss
888,889,0,3,female,NaN,1,2,23.4500,S,0,Miss
889,890,1,1,male,26.0,0,0,30.0000,C,1,Mr


In [20]:
#Train dataset and onwards
X_train = train.iloc[:,2:].values
y_train = train.iloc[:,1].values

In [21]:
X_train.shape

(889, 9)

In [22]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan,strategy='median')
imputer.fit(X_train[:,2:3])
(X_train[:,2:3]) = imputer.transform(X_train[:,2:3])

In [23]:
X_train

array([[3, 'male', 22.0, ..., 'S', 0, 'Mr'],
       [1, 'female', 38.0, ..., 'C', 1, 'Mrs'],
       [3, 'female', 26.0, ..., 'S', 0, 'Miss'],
       ...,
       [3, 'female', 28.0, ..., 'S', 0, 'Miss'],
       [1, 'male', 26.0, ..., 'C', 1, 'Mr'],
       [3, 'male', 32.0, ..., 'Q', 0, 'Mr']], dtype=object)

In [24]:
#Feature Encoding
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
#OneHotEncoder for  Sex,Embark,Title (in order)
ct_1 = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1,6,8])],remainder='passthrough')
X_train = np.array(ct_1.fit_transform(X_train))

In [25]:
X_train[0]

array([0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 3, 22.0, 1, 0,
       7.25, 0], dtype=object)

In [26]:
#Feature Scaling - standardize age (index 11) and normalise fare (index 14)
from sklearn.preprocessing import StandardScaler
sc_1 = StandardScaler()
X_train[:,11:12] = sc_1.fit_transform(X_train[:,11:12])
from sklearn.preprocessing import MinMaxScaler
sc_2 = StandardScaler()
mms = MinMaxScaler()
X_train[:,14:15] = sc_2.fit_transform(X_train[:,14:15])

In [27]:
#Test data clean
test['CabinMultiple'] = test['Cabin'].apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))
test['Title'] = test['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())
list_1 = test['Title'].value_counts().index
list_2 = list_1.difference(['Mr','Mrs','Miss','Master'])
test.drop(columns=['Name','Cabin','Ticket'],inplace=True)
test['Title'] = test['Title'].replace(list_2,'Others')
# test.dropna(subset=['Embarked','Fare'],inplace=True)
# train.Age = train.Age.fillna(train.Age.median)
#Train dataset and onwards
X_test = test.iloc[:,1:].values
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan,strategy='median')
imputer.fit(X_test[:,2:3])
(X_test[:,2:3]) = imputer.transform(X_test[:,2:3])
imputer = SimpleImputer(missing_values=np.nan,strategy='median')
imputer.fit(X_test[:,5:6])
(X_test[:,5:6]) = imputer.transform(X_test[:,5:6])
#Feature Encoding
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
#OneHotEncoder for  Sex,Embark,Title (in order)
ct_1 = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1,6,8])],remainder='passthrough')
X_test = np.array(ct_1.fit_transform(X_test))
#Feature Scaling - standardize age (index 11) and normalise fare (index 14)
from sklearn.preprocessing import StandardScaler
sc_1 = StandardScaler()
X_test[:,11:12] = sc_1.fit_transform(X_test[:,11:12])
from sklearn.preprocessing import MinMaxScaler
sc_2 = StandardScaler()
mms = MinMaxScaler()
X_test[:,14:15] = sc_2.fit_transform(X_test[:,14:15])

In [28]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
cf_DT = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
cf_DT.fit(X_train, y_train)
cf_DT_cvs = cross_val_score(estimator = cf_DT, X = X_train, y = y_train, cv = 10)
cf_DT_cvs.mean()
from sklearn.neighbors import KNeighborsClassifier
cf_KNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
cf_KNN.fit(X_train, y_train)
cf_KNN_cvs = cross_val_score(estimator = cf_KNN, X = X_train, y = y_train, cv = 10)
cf_KNN_cvs.mean()
from sklearn.svm import SVC
cf_SVC = SVC(kernel = 'rbf', random_state = 0,probability = True)
cf_SVC.fit(X_train, y_train)
cf_SVC_cvs = cross_val_score(estimator = cf_SVC, X = X_train, y = y_train, cv = 10)
cf_SVC_cvs.mean()
from sklearn.linear_model import LogisticRegression
LG = LogisticRegression(random_state = 0)
LG.fit(X_train, y_train)
LG_cvs = cross_val_score(estimator = LG, X = X_train, y = y_train, cv = 10)
LG_cvs.mean()
from sklearn.naive_bayes import GaussianNB
NB = GaussianNB()
NB.fit(X_train, y_train)
NB_cvs = cross_val_score(estimator = NB, X = X_train, y = y_train, cv = 10)
NB_cvs.mean()
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
RFC.fit(X_train, y_train)
RFC_cvs = cross_val_score(estimator = RFC, X = X_train, y = y_train, cv = 10)
RFC_cvs.mean()
# from sklearn.svm import SVC
# LinSupVec = SVC(kernel = 'linear', random_state = 0,probability = True)
# LinSupVec.fit(X_train, y_train)
# LinSupVec_cvs = cross_val_score(estimator = LinSupVec, X = X_train, y = y_train, cv = 10)
# LinSupVec_cvs.mean()
from xgboost import XGBClassifier
XGB = XGBClassifier(use_label_encoder=False)
XGB.fit(X_train, y_train)
XGB_cvs = cross_val_score(estimator = XGB, X = X_train, y = y_train, cv = 10)
XGB_cvs.mean()

[03:07:38] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:07:38] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:07:38] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:07:38] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

0.8155388151174667

In [29]:
df_basemean = {
    'Classifiers': ["Decision Tree","K-NN",'RBF Support Vector','Logistic Regression','Naive Bayes','Random Forest','XGB'],
    'Mean Scores': [cf_DT_cvs.mean(),cf_KNN_cvs.mean(),cf_SVC_cvs.mean(),LG_cvs.mean(),NB_cvs.mean(),RFC_cvs.mean(),XGB_cvs.mean()]
}
df_basemean = pd.DataFrame(df_basemean)
df_basemean

,Classifiers,Mean Scores
0,Decision Tree,0.788534
1,K-NN,0.822255
2,RBF Support Vector,0.834678
3,Logistic Regression,0.826826
4,Naive Bayes,0.791892
5,Random Forest,0.820046
6,XGB,0.815539


In [30]:
from sklearn.ensemble import VotingClassifier
vc = VotingClassifier(estimators = [("cf_DT",cf_DT),('cf_KNN',cf_KNN),('LG',LG),('NB',NB),('cf_SVC',cf_SVC),('RFC',RFC),('XGB',XGB)],voting='soft')
vc = vc.fit(X_train,y_train)
vc.predict(X_test).astype(int)
vc_cvs = cross_val_score(estimator = vc, X = X_train, y = y_train, cv = 10)
vc_cvs.mean()

[03:07:39] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:07:40] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:07:40] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:07:40] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

0.8369126659856997

In [34]:
from sklearn.model_selection import GridSearchCV
parameters = [{'C': [0.25,0.5,0.75,1], 'kernel':['linear']},
              {'C': [0.25,0.5,0.75,1], 'kernel':['rbf'], 'gamma':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]}]
grid_search = GridSearchCV(estimator=cf_SVC,
                           param_grid = parameters,
                           scoring='accuracy',
                           cv=10,
                           n_jobs = -1)
grid_search.fit(X_train,y_train)
best_estimator = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Accuracy: {:.2f} %".format(best_estimator.mean()*100))
print("Parameters: ", best_parameters)
SVC_best_estimator = best_estimator

Accuracy: 83.58 %
Parameters:  {'C': 0.25, 'gamma': 0.5, 'kernel': 'rbf'}


In [35]:
from sklearn.model_selection import GridSearchCV
parameters = [{'min_samples_split': [2,4,6], 
               'max_features': ['auto','sqrt','log2'],
               'max_depth': [1,2,4,8], 
               'criterion':['entropy','gini']}]
grid_search = GridSearchCV(estimator=cf_DT,
                           param_grid = parameters,
                           scoring='accuracy',
                           cv=10,
                           n_jobs = -1)
grid_search.fit(X_train,y_train)
best_estimator = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Accuracy: {:.2f} %".format(best_estimator.mean()*100))
print("Parameters: ", best_parameters)
cf_DT_best_estimator = best_estimator

Accuracy: 81.67 %
Parameters:  {'criterion': 'gini', 'max_depth': 4, 'max_features': 'auto', 'min_samples_split': 2}


In [36]:
from sklearn.model_selection import GridSearchCV
parameters = [{'n_neighbors': [5,7,8,10,15,20], 
#                'max_features': ['auto','sqrt','log2'],
               'weights':['uniform','distance'], 
               'metric':['minkowski','euclidean'],
               'p' : ['1','2']
              }]
grid_search = GridSearchCV(estimator=cf_KNN,
                           param_grid = parameters,
                           scoring='accuracy',
                           cv=10,
                           n_jobs = -1)
grid_search.fit(X_train,y_train)
best_estimator = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Accuracy: {:.2f} %".format(best_estimator.mean()*100))
print("Parameters: ", best_parameters)
cf_KNN_best_estimator = best_estimator

Accuracy: 82.23 %
Parameters:  {'metric': 'euclidean', 'n_neighbors': 5, 'p': '1', 'weights': 'uniform'}


In [37]:
from sklearn.model_selection import GridSearchCV
parameters = [{'C': [0.25,0.5,0.75,1], 
               'penalty':['l1','l2','elasticnet','none'], 
               'multi_class':['auto','ovr','multinomial'],
               'solver':['newton-cg','liblinear','lbfgs','sag','saga']}]
grid_search = GridSearchCV(estimator=LG,
                           param_grid = parameters,
                           scoring='accuracy',
                           cv=10,
                           n_jobs = -1)
grid_search.fit(X_train,y_train)
best_estimator = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Accuracy: {:.2f} %".format(best_estimator.mean()*100))
print("Parameters: ", best_parameters)
LG_best_estimator = best_estimator

Accuracy: 83.02 %
Parameters:  {'C': 0.75, 'multi_class': 'auto', 'penalty': 'l1', 'solver': 'liblinear'}


In [38]:
from sklearn.model_selection import GridSearchCV
parameters = [{'n_estimators': [100,200,400,800], 
               'criterion': ['gini','entropy'],
               'max_features':['auto','sqrt','log2'], 
               'min_samples_leaf':[2,4,6,8],
               'max_depth': [1,2,4,8],
               'bootstrap': [True] #This means that some values will be used multiple times for the model
              }]
grid_search = GridSearchCV(estimator=RFC,
                           param_grid = parameters,
                           scoring='accuracy',
                           cv=10,
                           n_jobs = -1)
grid_search.fit(X_train,y_train)
best_estimator = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Accuracy: {:.2f} %".format(best_estimator.mean()*100))
print("Parameters: ", best_parameters)
RFC_best_estimator = best_estimator

Accuracy: 83.92 %
Parameters:  {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 8, 'max_features': 'auto', 'min_samples_leaf': 4, 'n_estimators': 800}


In [ ]:
# from sklearn.model_selection import GridSearchCV
# parameters = {
#     'n_estimators': [450,500,550],
#     'colsample_bytree': [0.75,0.8,0.85],
#     'tree_method':['hist'],
#     'grow_policy':['depthwise','lossguide'],
#     'max_depth': [None],
#     'reg_alpha': [1],
#     'reg_lambda': [2, 5, 10],
#     'subsample': [0.55, 0.6, .65],
#     'learning_rate':[0.5],
#     'gamma':[.5,1,2],
#     'min_child_weight':[0.01],
#     'sampling_method': ['uniform']
# }
# grid_search = GridSearchCV(estimator=XGB,
#                            param_grid = parameters,
#                            scoring='accuracy',
#                            cv=10,
#                            n_jobs = -1)
# print('lol')
# grid_search.fit(X_train,y_train)
# best_estimator = grid_search.best_score_
# best_parameters = grid_search.best_params_
# print("Accuracy: {:.2f} %".format(best_estimator.mean()*100))
# print("Parameters: ", best_parameters)
# XGB_best_estimator = best_estimator

In [39]:
df_bestmean = {
    'Classifiers': ["Decision Tree","K-NN",'RBF Support Vector','Logistic Regression','Naive Bayes','Random Forest','XGB'],
    'Mean Scores': [cf_DT_best_estimator.mean(),cf_KNN_best_estimator.mean(),SVC_best_estimator.mean(),LG_best_estimator.mean(),NB_cvs.mean(),RFC_best_estimator.mean(),XGB_cvs.mean()]
}
df_bestmean = pd.DataFrame(df_bestmean)
df_bestmean

,Classifiers,Mean Scores
0,Decision Tree,0.816675
1,K-NN,0.822255
2,RBF Support Vector,0.835789
3,Logistic Regression,0.830197
4,Naive Bayes,0.791892
5,Random Forest,0.839160
6,XGB,0.815539


In [40]:
df_basemean.merge(df_bestmean,left_on='Classifiers',right_on='Classifiers')

,Classifiers,Mean Scores_x,Mean Scores_y
0,Decision Tree,0.788534,0.816675
1,K-NN,0.822255,0.822255
2,RBF Support Vector,0.834678,0.835789
3,Logistic Regression,0.826826,0.830197
4,Naive Bayes,0.791892,0.791892
5,Random Forest,0.820046,0.839160
6,XGB,0.815539,0.815539


In [41]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
cf_DT = DecisionTreeClassifier(criterion = 'gini', max_depth= 4, max_features= 'auto', min_samples_split= 2)
cf_DT.fit(X_train, y_train)
cf_DT_cvs = cross_val_score(estimator = cf_DT, X = X_train, y = y_train, cv = 10)
cf_DT_cvs.mean()
from sklearn.neighbors import KNeighborsClassifier
cf_KNN = KNeighborsClassifier(n_neighbors = 5, metric = 'euclidean', p = 1,weights = 'uniform')
cf_KNN.fit(X_train, y_train)
cf_KNN_cvs = cross_val_score(estimator = cf_KNN, X = X_train, y = y_train, cv = 10)
cf_KNN_cvs.mean()
from sklearn.svm import SVC
cf_SVC = SVC(C = 0.25, gamma = 0.5, kernel = 'rbf', probability = True)
cf_SVC.fit(X_train, y_train)
cf_SVC_cvs = cross_val_score(estimator = cf_SVC, X = X_train, y = y_train, cv = 10)
cf_SVC_cvs.mean()
from sklearn.linear_model import LogisticRegression
LG = LogisticRegression(C = 0.75, multi_class = 'auto', penalty = 'l1', solver = 'liblinear')
LG.fit(X_train, y_train)
LG_cvs = cross_val_score(estimator = LG, X = X_train, y = y_train, cv = 10)
LG_cvs.mean()
from sklearn.naive_bayes import GaussianNB
NB = GaussianNB()
NB.fit(X_train, y_train)
NB_cvs = cross_val_score(estimator = NB, X = X_train, y = y_train, cv = 10)
NB_cvs.mean()
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(n_estimators = 800, 
                             criterion = 'entropy', bootstrap = True, max_depth = 8, max_features = 'auto',
                            min_samples_leaf = 4)
RFC.fit(X_train, y_train)
RFC_cvs = cross_val_score(estimator = RFC, X = X_train, y = y_train, cv = 10)
RFC_cvs.mean()
# from sklearn.svm import SVC 
# LinSupVec = SVC(kernel = 'linear', random_state = 0,probability = True)
# LinSupVec.fit(X_train, y_train)
# LinSupVec_cvs = cross_val_score(estimator = LinSupVec, X = X_train, y = y_train, cv = 10)
# LinSupVec_cvs.mean()
from xgboost import XGBClassifier
XGB = XGBClassifier(use_label_encoder=False)
XGB.fit(X_train, y_train)
XGB_cvs = cross_val_score(estimator = XGB, X = X_train, y = y_train, cv = 10)
XGB_cvs.mean()

[04:00:50] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[04:00:51] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[04:00:51] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[04:00:51] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

0.8155388151174667

In [42]:
from sklearn.ensemble import VotingClassifier
vc_opt = VotingClassifier(estimators = [("cf_DT",cf_DT),('cf_KNN',cf_KNN),('LG',LG),('NB',NB),('cf_SVC',cf_SVC),('RFC',RFC),('XGB',XGB)],voting='soft')
vc_opt = vc.fit(X_train,y_train)
vc_opt.predict(X_test).astype(int)
# vc_cvs = cross_val_score(estimator = vc, X = X_train, y = y_train, cv = 10)
# vc_cvs.mean()

[04:00:52] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [43]:
y_pred = vc_opt.predict(X_train).astype(int)


In [44]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm=confusion_matrix(y_pred,y_train)
vc_opt_cvs = cross_val_score(estimator = vc, X = X_train, y = y_train, cv = 10)
print(vc_opt_cvs.mean())
print(cm)
accuracy_score(y_pred,y_train)

[04:00:52] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[04:00:53] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[04:00:53] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[04:00:53] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

0.9111361079865017

In [54]:
y_pred = vc_opt.predict(X_test).astype(int)
test['Survived'] = y_pred
test_predictions_2 = test.reindex(columns=['PassengerId','Survived'])
test_predictions_2

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
